## 一、多线程和多进程概述

当计算机运行程序时，就会创建包含有代码和状态的进程。这些进程会通过计算机的一个或多个CPU执行。不过，同一时刻每个CPU只会执行一个进程，然后在不同进程间快速切换，这样就给人以多个程序同时运行的感觉。同理，在一个进程中，程序的执行也是在不同线程间进行切换的，每个线程执行程序的不同部分。

## 二、多进程使用方法

Python进行多进程爬虫使用了multiprocessing库，本书使用multiprocessing库的进程池方法进行多进程爬虫，使用方法如下代码：


from multiprocessing import Pool

pool = Pool(processes=4)			#创建进程池

pool.map(func,iterable[,chunksize])

（1）第一行用于导入multiprocessing库的Pool模块。

（2）第二行用于创建进程池，processes参数为设置进程的个数。

（3）第三行利用map()函数进行进程运行，func参数为需运行的函数，在爬虫实战中，为爬虫函数。iterable为迭代参数，在爬虫实战中，可为多个URL列表进行迭代。


## 三、性能对比

多进程爬虫速度要远优于串行爬虫，通过下面的案例进行实证。

## 案例1-糗事百科

In [9]:
import requests
import re
import time
from multiprocessing import Pool

In [10]:
headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}

In [11]:
#定义获取信息的函数

In [12]:
def re_scraper(url):
    res=requests.get(url,headers=headers)
    ids = re.findall('<h2>(.*?)</h2>',res.text,re.S)
    contents = re.findall('<div class="content">.*?<span>(.*?)</span>',res.text,re.S)
    laughs = re.findall('<span class="stats-vote"><i class="number">(\d+)</i>',res.text,re.S)
    comments = re.findall('<i class="number">(\d+)</i> 评论',res.text,re.S)
    for ids,contents,laughs,comments in zip(ids,contents,laughs,comments):
        info={
            'id':ids,
            'content':contents,
            'laugh':laughs,
            'comments':comments
        }
        return info

In [13]:
if __name__=='__main__':
    urls=['https://www.qiushibaike.com/text/page/{}'.format(str(i)) for i in range(1,36)]
    start_1=time.time()
    for url in urls:
        re_scraper(url)
    end_1=time.time()
    print('串行爬虫',end_1-start_1)
    start_2=time.time()
    pool= Pool(processes=2)
    pool.map(re_scraper,urls)
    end_2=time.time()
    print('两个进程',end_2-start_2)
    
    start_3=time.time()
    pool=Pool(processes=4)
    pool.map(re_scraper,urls)
    end_3=time.time()
    print('四个进程',end_3-start_3)

串行爬虫 12.275916814804077
两个进程 6.943901062011719
四个进程 3.597336769104004


### 1、爬取的内容为简书“首页投稿”热评文章的信息（http://www.jianshu.com/c/bDHhpK）

### 2、网站分析

### 3、需要爬取的信息有：用户ID、文章发表日期、文章标题、文章内容、浏览量、评论数、点赞数和打赏数。

### 4、实现方式

## 案例2-简书网热评文章

In [51]:
import requests
from lxml import etree

In [52]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}

In [53]:
b='_ga=GA1.2.483841705.1588943243; __gads=ID=db5014ced974adee:T=1588943242:S=ALNI_Mbbj91LLTRJ0hcvwilZwAEhdHeGhg; __yadk_uid=EzRoiQDE5jhUadS1IyOnnIBgJjl4y0Tw; _gid=GA1.2.2012616477.1590933467; _m7e_session_core=7088aac59717d281e916dec2b51b1052; locale=zh-CN; Hm_lvt_0c0e9d9b1e7d617b3e6842e85b9fb068=1590984970,1590985964,1590990135,1590994582; read_mode=day; default_font=font2; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%22171f465c3585f3-0ba6dfc5d20b32-30667d00-1296000-171f465c359725%22%2C%22first_id%22%3A%22%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E7%9B%B4%E6%8E%A5%E6%B5%81%E9%87%8F%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%2C%22%24latest_referrer%22%3A%22%22%2C%22%24latest_utm_source%22%3A%22recommendation%22%2C%22%24latest_utm_medium%22%3A%22seo_notes%22%2C%22%24latest_utm_campaign%22%3A%22maleskine%22%2C%22%24latest_utm_content%22%3A%22note%22%2C%22%24latest_referrer_host%22%3A%22%22%7D%2C%22%24device_id%22%3A%22171f465c3585f3-0ba6dfc5d20b32-30667d00-1296000-171f465c359725%22%7D; signin_redirect=https%3A%2F%2Fwww.jianshu.com%2Fc%2FbDHhpK%3Forder_by%3Dcommented_at%26page%3D1; Hm_lpvt_0c0e9d9b1e7d617b3e6842e85b9fb068=1591015882'

In [54]:
line=b.split(';')

In [55]:
cookie={}
for i in line:
    key,value = i.split('=',1)
    cookie[key] = value

In [9]:
##爬取第一页

In [61]:
title=[]
abstract=[]
author=[]
score=[]

In [57]:
url = 'http://www.jianshu.com/c/bDHhpK?order_by=commented_at&page='+str(1)
html = requests.get(url,headers=headers,cookies=cookie)
selector = etree.HTML(html.text)

In [58]:
selector.xpath('//ul[@class="note-list"]/li/div[@class="content"]/a/text()')

['BT下载与用python轻松自建种子搜索引擎',
 'iOS NSDictionary(字典)~实现原理',
 '如何用Python和机器学习训练中文文本情感分类模型？',
 'iOS形变之CATransform3D',
 'iOS 多线程：『NSOperation、NSOperationQueue』详尽总结',
 '怒！我的原创文章发布后的两小时就被剽窃了',
 '妃子侍寝过程中能叫床吗？事前太监先将妃子脱光',
 '我们家不需要性教育',
 'Android RxJava：功能性操作符 全面讲解 ',
 '成人书单丨那些红着脸捂着眼也要看完的书']

In [59]:
title.extend(selector.xpath('//ul[@class="note-list"]/li/div[@class="content"]/a/text()'))
abstract.extend(selector.xpath('//ul[@class="note-list"]/div[@class="content"]/p[@class="abstract"]/text()'))
author.extend(selector.xpath('//ul[@class="note-list"]/div[@class="meta"]/a[1]/text()'))
score.extend(selector.xpath('//ul[@class="note-list"]/div[@class="content"]/div[@class="meta"]/span[@class="jsd-meta"]/text()'))

In [60]:
title

['iOS NSDictionary(字典)~实现原理',
 '如何用Python和机器学习训练中文文本情感分类模型？',
 'iOS形变之CATransform3D',
 'iOS 多线程：『NSOperation、NSOperationQueue』详尽总结',
 '怒！我的原创文章发布后的两小时就被剽窃了',
 '妃子侍寝过程中能叫床吗？事前太监先将妃子脱光',
 '我们家不需要性教育',
 'Android RxJava：功能性操作符 全面讲解 ',
 '成人书单丨那些红着脸捂着眼也要看完的书',
 '微信小程序之生成图片分享',
 '新垣结衣恋爱史大公开！她的历代男友是？',
 '我是如何坚持看原版英文小说的（几本小说推荐)',
 '中国古代同性恋记载——男同篇',
 '一个男妓的自白',
 '你的脚美吗？恋足癖的男人你究竟恋的是什么！',
 '如何写一个整洁好看的标题',
 '多数人都太污了，我试玩了30多个直播App之后吐了一口老血#DEMO8产品专辑#',
 '鸣人和雏田的婚后生活',
 'Kotlin搞起来——3.集合与它的操作符们',
 'Android神兵利器 | ChkBugReport输出网页版错误报告',
 '记笔记，手写与打字哪个更有效？',
 '并置：空间叙事的艺术',
 '囚禁、强暴、虐恋、乱伦，你能想到的变态情节全在这部电影里了',
 '推荐18部高大上的情色电影，部部经典，色而不淫',
 'ios通用链接 UniversalLink在微信打开app',
 '谈一谈“美学”',
 '我该如何发表一篇付费文章？',
 '如何获取WebView内容高度',
 '简友常见问题汇总',
 '【iOS开发】IAP开发的那些坑',
 'BT下载与用python轻松自建种子搜索引擎',
 'iOS NSDictionary(字典)~实现原理',
 '如何用Python和机器学习训练中文文本情感分类模型？',
 'iOS形变之CATransform3D',
 'iOS 多线程：『NSOperation、NSOperationQueue』详尽总结',
 '怒！我的原创文章发布后的两小时就被剽窃了',
 '妃子侍寝过程中能叫床吗？事前太监先将妃子脱光',
 '我们家不需要性教育',
 'Android RxJava：功能性操作符 全面讲解 ',
 '成

In [ ]:
##爬取N页

In [81]:
title=[]
abstract=[]
author=[]
score=[]

In [82]:
for i in range(4):
    url = 'http://www.jianshu.com/c/bDHhpK?order_by=commented_at&page='+str(i)
    html = requests.get(url,headers=headers,cookies=cookie)
    selector = etree.HTML(html.text)
    title.extend(selector.xpath('//ul[@class="note-list"]/li/div[@class="content"]/a/text()'))
    abstract.extend(selector.xpath('//ul[@class="note-list"]/li/div[@class="content"]/p/text()'))
    author.extend(selector.xpath('//ul[@class="note-list"]/li/div/div[@class="meta"]/a[@class="nickname"]/text()'))
    score.extend(selector.xpath('//ul[@class="note-list"]/li/div[@class="content"]/div[@class="meta"]/span/text()'))

In [90]:
score

['\n          ',
 ' 3.8\n        ',
 '\n          ',
 ' 0.3\n        ',
 '\n          ',
 ' 3.6\n        ',
 '\n          ',
 ' 0.7\n        ',
 '\n          ',
 ' 35.5\n        ',
 '\n          ',
 ' 70.7\n        ',
 '\n          ',
 ' 0.4\n        ',
 '\n          ',
 ' 24.6\n        ',
 '\n          ',
 ' 3.8\n        ',
 '\n          ',
 ' 13.4\n        ',
 '\n          ',
 ' 6.5\n        ',
 '\n          ',
 ' 0.9\n        ',
 '\n          ',
 ' 3.9\n        ',
 '\n          ',
 ' 0.2\n        ',
 '\n          ',
 ' 11.6\n        ',
 '\n          ',
 ' 0.5\n        ',
 '\n          ',
 ' 39.1\n        ',
 '\n          ',
 ' 0.1\n        ',
 '\n          ',
 ' 0.7\n        ',
 '\n          ',
 ' 0.2\n        ',
 '\n          ',
 ' 0.3\n        ',
 '\n          ',
 ' 3.0\n        ',
 '\n          ',
 ' 0.9\n        ',
 '\n          ',
 ' 1.8\n        ',
 '\n          ',
 ' 31.1\n        ',
 '\n          ',
 ' 859.6\n        ',
 '\n          ',
 ' 3.0\n        ',
 '\n          ',
 '

In [87]:
scorenew=[]

In [88]:
for i in range(len(score)):
    if score[i]!='\n':
        scorenew.append(score[i].strip())
    else:
        pass

In [89]:
scorenew

['',
 '3.8',
 '',
 '0.3',
 '',
 '3.6',
 '',
 '0.7',
 '',
 '35.5',
 '',
 '70.7',
 '',
 '0.4',
 '',
 '24.6',
 '',
 '3.8',
 '',
 '13.4',
 '',
 '6.5',
 '',
 '0.9',
 '',
 '3.9',
 '',
 '0.2',
 '',
 '11.6',
 '',
 '0.5',
 '',
 '39.1',
 '',
 '0.1',
 '',
 '0.7',
 '',
 '0.2',
 '',
 '0.3',
 '',
 '3.0',
 '',
 '0.9',
 '',
 '1.8',
 '',
 '31.1',
 '',
 '859.6',
 '',
 '3.0',
 '',
 '678.4']